In [1]:
import scanpy as sc
import numpy as np
import pandas as pd

adata1 = sc.read_h5ad('/public/home/syj/scpDeconv/dataset/raw_data/mouse_PDAC_spatial_proteomics.h5ad') # mouse_PDAC/intersected_spatial_proteomics.h5ad
adata1

/public/home/syj/miniconda3/envs/SpatialDec/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AnnData object with n_obs × n_vars = 108 × 2837
    obs: 'ID2', 'Spot', 'X', 'Y'
    uns: 'celltype_gd'
    obsm: 'spatial'

In [2]:
adata1.uns['celltype_gd']

,CancerCell,Immune,CAF
ID,,,
spot_1,0.196205,0.070423,0.733372
spot_2,0.352113,0.050078,0.597809
spot_3,0.232003,0.058881,0.709116
spot_4,0.322770,0.205203,0.472027
spot_5,0.364828,0.095657,0.539515
...,...,...,...
spot_106,0.282864,0.091745,0.625391
spot_107,0.349374,0.053599,0.597027
spot_108,0.386933,0.056142,0.556925


In [3]:
adata2 = sc.read_h5ad('/public/home/syj/scpDeconv/dataset/raw_data/mouse_PDAC_reference_proteomics.h5ad') # mouse_PDAC/intersected_reference_proteomics.h5ad
adata2

AnnData object with n_obs × n_vars = 50 × 2837
    obs: 'Group', 'Lineage', 'celltype', 'celltype3'

In [4]:
adata2.obs['celltype3']

PCC_1     CancerCell
PCC_2     CancerCell
PCC_3     CancerCell
PCC_4     CancerCell
PCC_5     CancerCell
CAF_1            CAF
CAF_2            CAF
CAF_3            CAF
CAF_4            CAF
CAF_5            CAF
T4_1          Immune
T4_2          Immune
T4_3          Immune
T4_4          Immune
T4_5          Immune
T8_1          Immune
T8_2          Immune
T8_3          Immune
T8_4          Immune
T8_5          Immune
Treg_1        Immune
Treg_2        Immune
Treg_3        Immune
Treg_4        Immune
Treg_5        Immune
B_1           Immune
B_2           Immune
B_3           Immune
B_4           Immune
B_5           Immune
NEU_1         Immune
NEU_2         Immune
NEU_3         Immune
NEU_4         Immune
NEU_5         Immune
MO_1          Immune
MO_2          Immune
MO_3          Immune
MO_4          Immune
MO_5          Immune
MAC_1         Immune
MAC_2         Immune
MAC_3         Immune
MAC_4         Immune
MAC_5         Immune
DC_1          Immune
DC_2          Immune
DC_3         

In [5]:
sc.pp.normalize_total(adata1, target_sum=1e4)
sc.pp.log1p(adata1)
sc.pp.normalize_total(adata2, target_sum=1e4)
sc.pp.log1p(adata2)

In [6]:
x1 = adata2.X.A
y1 = adata2.obs['celltype3']
celltypes = np.unique(y1)

x, y = None, None
celltype_num = len(celltypes)
cells_sub = []

for i in range(celltype_num):
    cells_sub.append(x1[np.array(y1 == celltypes[i]), :])

for i in range(1000):

    # Create fractions for available celltypes
    fracs = np.random.rand(celltype_num)
    fracs_sum = np.sum(fracs)
    fracs = np.divide(fracs, fracs_sum)

    samp_fracs = np.multiply(fracs, 15)
    samp_fracs = np.array(list(map(round, samp_fracs)))

    artificial_samples = None

    for j in range(celltype_num):
        cells_fraction = np.random.randint(0, cells_sub[j].shape[0], samp_fracs[j])
        cells_choice = cells_sub[j][cells_fraction, :]
        if cells_choice.shape[0] > 0:
            artificial_samples = np.concatenate((artificial_samples, cells_choice)) if artificial_samples is not None else cells_choice
            

    cell = artificial_samples.mean(0).reshape(-1, 1)
    cell_y = (samp_fracs/samp_fracs.sum()).reshape(-1, 1)
    
    x = np.concatenate((x, cell), axis=1) if x is not None else cell
    y = np.concatenate((y, cell_y), axis=1) if y is not None else cell_y
    if i % 1000 == 0: print(i)

print(x.shape, y.shape)

0
(2837, 1000) (3, 1000)


In [7]:
df = pd.DataFrame(x.T, columns=adata2.var_names)
df

pid,A2A432,A2A863,A2A8U2,A2AAY5,A2ADY9,A2AGT5,A2AIV8,A2AKG8,A2AL36,A2AN08,...,Q9Z2U1,Q9Z2V5,Q9Z2W0,Q9Z2X1,Q9Z2X2,Q9Z2X8,Q9Z2Y8,Q9Z2Z6,Q9Z2Z9,Q9Z329
0,0.710046,1.081088,0.556259,0.925851,0.557684,0.712160,0.594207,0.533673,0.496700,1.156250,...,1.488435,0.489677,1.429067,2.948350,0.580743,0.640803,1.168193,1.340900,1.029808,0.945958
1,0.745993,0.948143,0.629841,0.826438,0.690685,0.746989,0.621935,0.500210,0.494952,1.110404,...,1.606569,0.541119,1.404617,3.027557,0.564689,0.546412,1.165141,1.268126,0.740254,0.959340
2,0.655082,1.137638,0.556221,0.934621,0.687390,0.656012,0.658013,0.564750,0.492963,1.166798,...,1.534232,0.480392,1.362532,3.009721,0.536400,0.619362,1.166043,1.418969,0.887641,0.961912
3,0.699715,1.027929,0.710134,0.908042,0.581776,0.785752,0.515610,0.489443,0.536591,1.184718,...,1.538913,0.553245,1.364053,3.006213,0.587598,0.604203,1.132913,1.392945,0.949436,0.974187
4,0.690487,1.068712,0.580823,0.962390,0.617885,0.762385,0.553564,0.524290,0.530931,1.140673,...,1.507414,0.537888,1.422281,2.906409,0.613021,0.617502,1.173410,1.350315,0.860667,1.004171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.742362,0.959992,0.683408,0.945476,0.519781,0.638120,0.526864,0.531196,0.505520,1.123594,...,1.501014,0.520987,1.429318,2.797156,0.565733,0.560597,1.260018,1.429814,1.024146,0.989105
996,0.730981,0.827936,0.783634,0.925659,0.679959,0.771664,0.606461,0.458579,0.512987,1.130967,...,1.558120,0.551427,1.348558,2.884602,0.536727,0.529033,1.141955,1.333938,1.011734,0.970891
997,0.737334,1.028796,0.719622,0.919958,0.595658,0.705652,0.610502,0.512437,0.500474,1.188437,...,1.538030,0.548603,1.392574,2.967534,0.553112,0.650821,1.186450,1.355515,0.991381,0.959808
998,0.704084,1.043452,0.640896,0.877823,0.673394,0.763397,0.712460,0.526218,0.498251,1.138521,...,1.613412,0.535941,1.424957,2.989958,0.540431,0.619192,1.230711,1.280036,0.846493,0.950271


In [8]:
adata_0 = sc.AnnData(df)
for i in range(len(celltypes)):
    adata_0.obs[celltypes[i]] = y[i].T
adata_0.obs['batch'] = np.array(['0'] * adata_0.shape[0])
adata_0.uns['cell_types'] = celltypes

adata_0

/public/home/syj/miniconda3/envs/SpatialDec/lib/python3.9/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 1000 × 2837
    obs: 'CAF', 'CancerCell', 'Immune', 'batch'
    uns: 'cell_types'

In [9]:
df2 = pd.DataFrame(adata1.X.A, columns=adata1.var_names)
df2

pid,A2A432,A2A863,A2A8U2,A2AAY5,A2ADY9,A2AGT5,A2AIV8,A2AKG8,A2AL36,A2AN08,...,Q9Z2U1,Q9Z2V5,Q9Z2W0,Q9Z2X1,Q9Z2X2,Q9Z2X8,Q9Z2Y8,Q9Z2Z6,Q9Z2Z9,Q9Z329
0,0.184064,0.695723,0.096068,0.164779,0.384641,0.344744,0.098651,1.382003,1.147573,2.036255,...,1.547188,0.262859,0.838079,1.021841,0.052531,0.469385,0.202042,0.077309,0.934228,0.101838
1,0.237925,0.595428,0.099414,0.699611,0.864160,0.316363,0.052195,0.584249,1.224533,1.707355,...,1.152480,0.596934,0.876832,1.161506,0.525879,0.175316,0.161291,0.451001,0.802353,0.583627
2,0.053705,0.747485,0.149840,0.614269,0.630558,2.196714,0.084137,0.884090,1.008612,1.478622,...,0.940686,0.064587,0.815146,0.967838,0.047310,0.263184,0.117266,0.061655,0.678648,0.708926
3,0.510199,0.349212,0.154010,0.116954,0.088089,2.220500,0.194668,1.066941,1.466419,2.938432,...,0.968343,0.232514,0.177456,0.277701,0.128991,0.189437,0.192186,0.093833,0.132932,0.240019
4,0.087439,0.783951,0.085087,0.289586,0.059534,1.770212,0.099289,1.069842,1.189421,1.728656,...,1.062690,0.068997,0.778156,0.957955,0.044872,0.396037,0.110157,0.396878,0.733846,0.155029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.052502,0.757997,0.245060,0.117241,0.105728,0.241057,0.047492,2.016191,0.945964,1.402770,...,2.254956,0.091939,0.627977,0.734569,0.267326,0.256174,0.079652,0.140077,0.659474,0.633535
104,0.091563,0.310967,0.041627,0.340058,0.119598,0.202859,0.103494,0.595232,1.161638,1.299322,...,1.171708,0.083924,0.777758,0.619215,0.108118,0.142340,0.204067,0.295581,0.844354,0.593259
105,0.194282,0.598535,0.074696,0.461633,0.174039,0.151847,0.380499,0.918995,1.142507,1.152274,...,1.607446,0.168100,0.471009,0.683885,0.166734,0.264423,0.201246,0.101961,0.571410,0.329276
106,0.060496,0.714081,0.174568,0.109215,0.128872,0.056395,0.265043,0.688576,1.396496,0.961200,...,1.428637,0.177905,0.622190,0.589887,0.094982,0.404084,0.103621,0.059181,1.017845,0.113527


In [10]:
adata_1 = sc.AnnData(df2)
for i in range(len(celltypes)):
    adata_1.obs[celltypes[i]] = adata1.uns['celltype_gd'][celltypes[i]].values
adata_1.obs['batch'] = np.array(['1'] * adata_1.shape[0])
adata_1.uns['cell_types'] = celltypes

adata_1

/public/home/syj/miniconda3/envs/SpatialDec/lib/python3.9/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 108 × 2837
    obs: 'CAF', 'CancerCell', 'Immune', 'batch'
    uns: 'cell_types'

In [11]:
adata_1.obs['CAF'] 

0      0.733372
1      0.597809
2      0.709116
3      0.472027
4      0.539515
         ...   
103    0.625391
104    0.597027
105    0.556925
106    0.605634
107    0.788928
Name: CAF, Length: 108, dtype: float64

In [12]:
adata_0.var_names_make_unique()
adata_1.var_names_make_unique()
adata = sc.concat((adata_0, adata_1))
adata.uns['cell_types'] = celltypes
adata.uns['spatial'] = adata1.obsm['spatial']
adata

/public/home/syj/miniconda3/envs/SpatialDec/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 1108 × 2837
    obs: 'CAF', 'CancerCell', 'Immune', 'batch'
    uns: 'cell_types', 'spatial'

In [13]:
adata.write_h5ad('/public/home/syj/scpDeconv/dataset/mixup_data/mouse_PDAC.h5ad') # Please modify this to your address.